In [1]:
import pandas as pd
import numpy as np

# What are the files in the dataset?

In [2]:
!ls ecommerce-dataset/

category_tree.csv         item_properties_part1.csv
events.csv                item_properties_part2.csv


# Inspect events data

In [3]:
events_df = pd.read_csv('ecommerce-dataset/events.csv')
events_df.head()

,timestamp,visitorid,event,itemid,transactionid
0,1433221332117,257597,view,355908,NaN
1,1433224214164,992329,view,248676,NaN
2,1433221999827,111016,view,318965,NaN
3,1433221955914,483717,view,253185,NaN
4,1433221337106,951259,view,367447,NaN


In [4]:
events_df.nunique()

timestamp        2750455
visitorid        1407580
event                  3
itemid            235061
transactionid      17672
dtype: int64

In [5]:
# events distribution
events_df.event.value_counts()

view           2664312
addtocart        69332
transaction      22457
Name: event, dtype: int64

# Inspect item properties data

In [6]:
item_df = pd.read_csv('ecommerce-dataset/item_properties_part1.csv')
item_df.head()

,timestamp,itemid,property,value
0,1435460400000,460429,categoryid,1338
1,1441508400000,206783,888,1116713 960601 n277.200
2,1439089200000,395014,400,n552.000 639502 n720.000 424566
3,1431226800000,59481,790,n15360.000
4,1431831600000,156781,917,828513


In [7]:
item_df = pd.concat( [item_df, pd.read_csv('ecommerce-dataset/item_properties_part2.csv')], axis=0)
item_df.head()

,timestamp,itemid,property,value
0,1435460400000,460429,categoryid,1338
1,1441508400000,206783,888,1116713 960601 n277.200
2,1439089200000,395014,400,n552.000 639502 n720.000 424566
3,1431226800000,59481,790,n15360.000
4,1431831600000,156781,917,828513


In [8]:
item_df.nunique()

timestamp         18
itemid        417053
property        1104
value        1966868
dtype: int64

In [9]:
# filter out the items not in events df
item_df = item_df.loc[item_df.itemid.isin(events_df.itemid)]
item_df.nunique()

timestamp         18
itemid        185246
property        1098
value        1000100
dtype: int64

In [10]:
# keep only the latest snapshot
item_df = item_df.drop_duplicates(subset=['itemid','property'], keep='last')
item_df.nunique()

timestamp        18
itemid       185246
property       1098
value        932001
dtype: int64

## Let's see what are the most common properties

In [11]:
item_df.property.value_counts().to_frame().head(10)

,property
283,185246
112,185246
available,185246
764,185246
159,185246
888,185246
364,185246
categoryid,185246
790,185246
678,185243


## Can we guess what they represent?

In [12]:
# every item's 764 property has the same value
item_df.loc[(item_df.property=='764')].value.value_counts().to_frame()

,value
1285872,185246


In [13]:
# every item's 159 property has the same value
item_df.loc[(item_df.property=='159')].value.value_counts().to_frame()

,value
519769,185246


In [14]:
item_df.loc[item_df.property=='available'].value.value_counts().to_frame()

,value
0,139182
1,46064


In [15]:
item_df.loc[item_df.property=='categoryid'].value.value_counts().to_frame()

,value
342,4697
769,3977
1483,3001
1051,2557
1007,2174
1680,2125
242,2100
959,1890
209,1662
196,1659


In [16]:
item_df.loc[item_df.property=='364'].value.value_counts().to_frame()

,value
143136,2
560583,2
418587,2
221208,2
355129,1
802229,1
566742,1
1222133,1
914978,1
944239,1


In [17]:
# 888 can be item descriptions/ titles
item_df.loc[item_df.property=='888'].value.value_counts().to_frame()

,value
769062,32
153900,25
138668,25
326996,24
92955,22
1177237,22
373953,20
1173998,20
1012579 448942,19
215273,19


In [18]:
item_df.loc[(item_df.property=='790')].value.value_counts().to_frame().head(10)

,value
n0.000,1769
n8400.000,706
n9480.000,494
n8280.000,479
n7800.000,478
n11880.000,448
n8520.000,443
n14280.000,443
n7080.000,431
n10680.000,427


In [19]:
# 790 might be price
item_df.loc[(item_df.property=='790')].value.apply(lambda x:x[0]!='n').sum()

0

In [20]:
item_df.loc[(item_df.property=='678')].value.value_counts().to_frame()

,value
820477,4582
286484,2970
212349,2745
367680,2558
1115724,2450
202346,2415
1318713,2118
896095,2086
1101877,2046
127431,1687


In [21]:
# 678 might be country
item_df.loc[(item_df.property=='678')].value.nunique()

3448

# Consider only the top useful properties

In [22]:
item_df = item_df.loc[item_df.property.isin(['categoryid','888','790','678'])]
item_df.head()

,timestamp,itemid,property,value
0,1435460400000,460429,categoryid,1338
16,1435460400000,178601,790,n5400.000
40,1433646000000,152892,888,599031
42,1431831600000,125874,790,n39588.000
46,1433646000000,272201,790,n10320.000
